# BSDE in 1D : example of an European Option

In [1]:
%pylab inline --no-import-all
import numpy as np
import scipy as sp
import pylab as pl
import matplotlib.pyplot as plt
import statistics as st
from scipy.stats import norm
from sklearn.ensemble import RandomForestRegressor

Populating the interactive namespace from numpy and matplotlib


In [2]:
def generate_paths(S0, dt , sigma, r, N, m,q):
    '''
    Parameters
    ==========
    S0    : float
            positive, initial Stock Value
    dt    : float
            time step discretization
    sigma : float 
            volatility 
    r     : float
            bond return 
    N     : int 
            Number of paths generated      
    m     : int 
            number of steps
    
    Returns
    =======
    S : matrix
        generated paths
    '''
    S = np.zeros((m + 1, N))
    S[0] = S0
    for t in range(1, m + 1):
        rand = np.random.standard_normal(N)
        S[t] = S[t - 1] * np.exp((r -q- sigma ** 2 / 2) * dt + sigma * dt ** 0.5 * rand)
    return S

In [3]:
def exercise_matrix(K,S): 
    h=np.maximum(K-S,0)
    return h

In [4]:
                                            "Pricing of the American Option"



def pricing_option(K,r,sigma,N,m,S_0,T, RF_n_estimators,RF_max_leaf_nodes, q):
  
    #Time-step
    dt=T/m    
    #Discount factor
    df=1/(1+r*dt)
    #Creation of a stock path matrix
    S = generate_paths(S_0, dt , sigma, r, N, m,q)
    #price of the option at time T = Initialization for a put
    Y=np.maximum(K-S[-1],0)
    #Initialization of the Price by Monte Carlo
    Y_opt=0 
       
    #Iteration over time"
    for t in range (m-1 ,0,-1): 
        X=S[t]
        X=X[:,None]    
        rf=RandomForestRegressor(n_estimators=RF_n_estimators,
                                max_leaf_nodes=RF_max_leaf_nodes,oob_score=False, n_jobs=-1)
        rf.fit(X,Y)
        Y=rf.predict(X)*df
    
    Y_opt= df*sum(Y)/N
    return (Y_opt)

In [5]:
"""Pricing with BS formula"""

def black_scholes(K,r,sigma, S_0,T): 
    d1=1/(sigma*np.sqrt(T))*(np.log(S_0/K)+(r+0.5*sigma*sigma)*T)
    d2=d1-sigma*np.sqrt(T)
    dscnt=np.exp(-r*T)
    Put=norm.cdf(-d2)*K*dscnt-norm.cdf(-d1)*S_0
    return (Put)

In [7]:
                                """Pricing by Random Forest Regressor"""

"Parameters"
T=1.
m=12
K=100.
S_0=100.
r=0.01
sigma=0.3
N=10000
RF_n_estimators = 100
RF_max_leaf_nodes = 50
q = 0.

price_BSDE=pricing_option(K,r,sigma,N,m,S_0,T, RF_n_estimators,RF_max_leaf_nodes, q)
price_BScholes=black_scholes(K,r,sigma, S_0,T)

print ("price ", price_BSDE) 
print ("price BScholes ", price_BScholes)

price  11.2471090878
price BScholes  11.3732508387
